In [10]:
# Importing the libraries needed
import pandas as pd
import numpy as np
import json
import scipy 
import torch
from torch.utils.data import Dataset, DataLoader
import logging

In [6]:
# Data loading 
file_path = "/Users/olgapodolska/Desktop/UA_War_my/resources/aug_twitter_data_cleaned.csv"
eng_df = pd.read_csv(file_path)

/var/folders/cw/wcg51kjd6qvc1klqs9sykxb40000gn/T/ipykernel_7340/303370205.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  eng_df = pd.read_csv(file_path)


In [3]:
# show the first 5 rows of the august dataframe
eng_df.head(5)

,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags,language,favorite_count,is_retweet,is_quote_status,extractedts,cleaned_text
0,51,3362,71331,2011-09-08 02:34:54.000000,2022-08-01 00:00:00.000000,0,The #RussiaUkraine conflict is being cast in b...,"[{'text': 'RussiaUkraine', 'indices': [4, 18]}...",en,0.0,False,False,2022-08-01 02:30:13.601013,the conflict is being cast in binaries making ...
1,2964,4669,119795,2017-07-06 15:28:29.000000,2022-08-01 00:00:00.000000,560,Remember when #NATO smashed #Yugoslavia into s...,"[{'text': 'NATO', 'indices': [14, 19]}, {'text...",en,0.0,True,False,2022-08-01 02:30:13.573250,remember when smashed into seven fragments and...
2,49,8,112,2021-04-04 17:47:00.000000,2022-08-01 00:00:01.000000,0,"Hi #NATO , you have to understand that we are ...","[{'text': 'NATO', 'indices': [3, 8]}]",en,0.0,False,False,2022-08-01 06:59:10.496078,hi you have to understand that we are billion...
3,137,114789,315096,2010-08-20 11:19:22.000000,2022-08-01 00:00:01.000000,0,The world is in dire straits as it is not equi...,"[{'text': 'UkraineCrisis', 'indices': [133, 14...",en,1.0,False,False,2022-08-01 01:44:58.884242,the world is in dire straits as it is not equi...
4,27,3059,13665,2017-01-06 10:17:40.000000,2022-08-01 00:00:01.000000,0,Will the #sanctions imposed on #Russia cause a...,"[{'text': 'sanctions', 'indices': [9, 19]}, {'...",en,0.0,False,False,2022-08-01 05:13:47.957650,will the imposed on cause a significant shift ...


In [4]:
# get shape of the DataFrame
print(f"{eng_df.shape[0]} rows and {eng_df.shape[1]} columns")

1255638 rows and 14 columns


In [5]:
# Check data types
eng_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255638 entries, 0 to 1255637
Data columns (total 14 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   following        1124566 non-null  object 
 1   followers        1124506 non-null  object 
 2   totaltweets      1124506 non-null  object 
 3   usercreatedts    1124506 non-null  object 
 4   tweetcreatedts   1124506 non-null  object 
 5   retweetcount     1124506 non-null  object 
 6   text             1124506 non-null  object 
 7   hashtags         1050085 non-null  object 
 8   language         1050058 non-null  object 
 9   favorite_count   1050058 non-null  float64
 10  is_retweet       1050058 non-null  object 
 11  is_quote_status  1050058 non-null  object 
 12  extractedts      1050058 non-null  object 
 13  cleaned_text     1049977 non-null  object 
dtypes: float64(1), object(13)
memory usage: 134.1+ MB


# Sentiment Analysis Using RoBERTa

For each tweet the RoBERTa model will generate a score for each of negative, neutral, and positive sentiments

In [2]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Number of GPUs: {torch.cuda.device_count()}")

Number of GPUs: 0


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [4]:
from transformers import AutoModelForSequenceClassification

# Create instance of twitter-roberta-base-sentiment classification model
#model = AutoModel.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

In [5]:
# Attach it to the cuda
model = model.to(device)

In [7]:
import urllib
import csv

labels=[] # will contain 'positive', 'neutral', 'negative'
task = 'sentiment' # our task is sentiment analysis
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [8]:
labels

['negative', 'neutral', 'positive']

In [11]:
from scipy.special import softmax
from tqdm import tqdm

BATCH_SIZE = 100 # number of tweets in a batch that will be passed into tokenizer

scores_all = np.empty((0,len(labels)))
# create list of all the tweets in the dataset
text_all = eng_df['cleaned_text'].to_list()
n = len(text_all) # same as number of tweets
with torch.no_grad():
    for start_idx in tqdm(range(0, n, BATCH_SIZE)):
        end_idx = min(start_idx + BATCH_SIZE, n) 
        # reference: https://huggingface.co/docs/transformers/preprocessing
        # tokenize the tweets in the batch, return pytorch ('pt') tensors
        # some tweets are shorter than the uniform tensor length needed; padding adds 0's to maintain uniform tensor length
        # some tweets are too long; truncation truncates input to maximum length accepted by model
        encoded_input = tokenizer(text_all[start_idx:end_idx], return_tensors='pt', padding=True, truncation=True).to(device)
        
        # references: https://stackoverflow.com/questions/11315010/what-do-and-before-a-variable-name-mean-in-a-function-signature
        # https://stackoverflow.com/questions/1419046/normal-arguments-vs-keyword-arguments/1419160#1419160
        output = model(**encoded_input)
        # convert pytorch tensor to numpy
        scores = output[0].detach().cpu().numpy()
        # 
        scores = softmax(scores, axis=1)
        scores_all = np.concatenate((scores_all, scores), axis=0)
        
        # delete encoded_input, output, scores for next batch
        del encoded_input, output, scores 
        # release all unoccupied cached mem 
        torch.cuda.empty_cache()

  0%|          | 57/12557 [17:01<62:14:01, 17.92s/it]


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

Output below is what scores_all looks like. Each row contains scores for negative, neutral, and positive sentiments. The higher the score, the more likely the tweet has that sentiment.

In [ ]:
scores_all

In [ ]:
# Let's combine the scores with the existing DataFrame.
eng_df[labels] = pd.DataFrame(scores_all, columns=labels)
eng_df.head()

In [ ]:
logging.basicConfig(level=logging.ERROR)

In [36]:
# Save this DataFrame so that we don't have to run the model again, which takes a long time.
file_path = "resources/aug_twitter_sentiments.csv"
eng_df.to_csv(file_path, index=False)